In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from itertools import islice
from mard.brazilian_document.oab import oab_pattern, oab_parser
from pathlib import Path
from bs4 import BeautifulSoup
import json
import re


next_file = next(islice(Path('/home/mard/attachments').glob('*.json'), 7, None))
with next_file.open() as f:
    obj = json.load(f)
    soup = BeautifulSoup(obj['content'])
    
    print(oab_parser._pattern.pattern)
    
    for x in oab_parser.parse(soup.text):
        print(x)
    

ModuleNotFoundError: No module named 'bs4'

In [ ]:
from itertools import islice
from mard.lazy import as_chunks
from mard.concurrent import pipeline
from tqdm.auto import tqdm
from random import sample
import re


def producer():
    def get_files():
        files = Path('/home/mard/attachments').glob('*.json')
        for file in files:
            with file.open() as f:
                yield f.read()
    yield from as_chunks(get_files(), 100)
    
def process_one(obj: dict):
    text = BeautifulSoup(obj['content'], features='lxml').text
    return list(oab_parser.parse(text))

def mapper(chunk):
    return [
        process_one(json.loads(x))
        for x in chunk
    ]

results = (
    result
    for chunk in pipeline(producer, mapper)
    for result in chunk
)

findings = []
for result in tqdm(results):
    for finding in result:
        findings.append(finding)

In [34]:
findings[22]

'381587/SP'